Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [405]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
from tqdm import tqdm
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [406]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [407]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

## First Greedy Algorithm

In [408]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
#FROM the starting city, that is the first city, we will find the closest city and add it to the path, iteratively
# until all cities are visited
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))
logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
)

tsp_greedy1 = tsp.copy()

tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Bologna (43.43km)
DEBUG:root:step: Bologna -> Modena (37.29km)
DEBUG:root:step: Modena -> Reggio nell'Emilia (23.94km)
DEBUG:root:step: Reggio nell'Emilia -> Parma (26.94km)
DEBUG:root:step: Parma -> Piacenza (57.65km)
DEBUG:root:step: Piacenza -> Milan (60.65km)
DEBUG:root:step: Milan -> Monza (14.51km)
DEBUG:root:step: Monza -> Bergamo (33.92km)
DEBUG:root:step: Bergamo -> Brescia (46.02km)
DEBUG:root:step: Brescia -> Verona (61.42km)
DEBUG:root:step: Verona -> Vicenza (44.70km)
DEBUG:root:step: Vicenza -> Padua (30.13km)
DEBUG:root:step: Padua -> Venice (36.07km)
DEBUG:root:step: Venice -> Trieste (115.09km)
DEBUG:root:step: Trieste -> Bolzano (209.68km)
DEBUG:root:step: Bolzano -> Trento (49.94km)
DEBUG:root:step: Trento -> Novara (206.69km)
DEBUG:root:step: Novara -> Turin (84.46

## Helper functions

In [409]:
#GENOTYPE: a list of integers, each representing a city
#PHENOTYPE: the path that visits all cities in the order specified by the genotype

def genotype_to_phenotype(genotype):
    return [CITIES.at[city, 'name'] for city in genotype]

def random_genotype():
    return np.random.permutation(len(CITIES))

# NEW TSP COST FUNCTION, no need to check if the path is closed
def tsp_cost(tsp):
    assert set(tsp) == set(range(len(CITIES)))
    #assert no duplicates
    assert len(tsp) == len(set(tsp))
    
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    tot_cost += DIST_MATRIX[tsp[-1], tsp[0]]
    return tot_cost

#MUTATION FUNCTIONS

def single_swap_mutation(genotype):
    i, j = np.random.choice(len(genotype), 2, replace=False)
    genotype[i], genotype[j] = genotype[j], genotype[i]
    return genotype

def multi_swap_mutation(genotype, p=0.8):
    first = True
    if first or np.random.rand() < p:
        i, j = np.random.choice(len(genotype), 2, replace=False)
        genotype[i], genotype[j] = genotype[j], genotype[i]
        first = False
    return genotype

def print_tsp(tsp, top_cost):
    logging.info(f"result: Found a path of {len(tsp)} steps, total length {top_cost:.2f}km")

    # Print path
    for city, closest in zip(tsp, tsp[1:]):
        logging.debug(
            f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
        )
    logging.debug(
        f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)"
    )

#For Greedy2
def arrange_segments(segments: list[tuple]) -> list[tuple]:
    connections = {}
    for u, v in segments:
        connections.setdefault(u, []).append(v)
        connections.setdefault(v, []).append(u)

    ordered_segments = [segments[0]]
    current = segments[0][1]

    while len(ordered_segments) < len(segments):
        next_city = connections[current].pop()
        connections[next_city].remove(current)
        ordered_segments.append((current, next_city))
        current = next_city

    return ordered_segments

def segments_to_path(segments: list[tuple]) -> list[int]:
    return [segment[0] for segment in segments]

def finalize_segment_tsp(segments: list[tuple], degrees: list[int]) -> list[tuple]:
    assert len(degrees) == len(CITIES), f"len(degrees)={len(degrees)} != len(CITIES)={len(CITIES)}"
    assert all(degree > 0 for degree in degrees), f"degrees={degrees} should be all > 0"

    endpoints = [index for index, degree in enumerate(degrees) if degree == 1]
    assert len(endpoints) == 2, f"len(endpoints)={len(endpoints)} != 2"
    segments.append((endpoints[0], endpoints[1]))

    return segments

    return tsp

NSTEPS = 10000

NSTEPS = 10000


## Greedy 2 : Shortest segment

In [410]:
# Create a list of all possible city pairs with their distances
city_pairs = list(combinations(range(len(CITIES)), 2))
city_pairs = [((i, j), DIST_MATRIX[i, j]) for i, j in city_pairs]

# Initialize the union-find structure and other variables
union_find = nx.utils.UnionFind()
tour = []
city_degrees = [0] * len(CITIES)

# Flag to check if all segments are processed
segments_processed = False
while not segments_processed:
    # Find and remove the shortest segment
    shortest_segment = min(city_pairs, key=lambda x: x[1])
    city_pairs.remove(shortest_segment)
    shortest_segment = shortest_segment[0]

    # Check if the cities are in different sets and have less than 2 connections
    different_sets = union_find[shortest_segment[0]] != union_find[shortest_segment[1]]
    less_than_two_connections = city_degrees[shortest_segment[0]] < 2 and city_degrees[shortest_segment[1]] < 2
    if different_sets and less_than_two_connections:
        # Union the sets and add the segment to the tour
        union_find.union(*shortest_segment)
        tour.append(shortest_segment)
        # Update the degrees of the cities
        city_degrees[shortest_segment[0]] += 1
        city_degrees[shortest_segment[1]] += 1

    segments_processed = len(city_pairs) == 0

# Finalize the tour by closing the cycle and arranging the segments
tour = finalize_segment_tsp(tour, city_degrees)
tour = arrange_segments(tour)

# Convert the segments to a list of cities and calculate the total cost
tour = segments_to_path(tour)
total_cost = tsp_cost(tour)
tsp_greedy2 = tour.copy()


Christofied algorithm

## Hill Climbing with Mutation and Simulated Annealing


Simulated Annealing

In [ ]:
def simulated_annealing(solution, full_history, bettering_history, NSTEPS, mutation_prob):
    tweak = multi_swap_mutation
    top_cost = -tsp_cost(solution)
    last_bettering = 0
    top_solution = solution.copy()
    for _ in range(int(NSTEPS)):
        new_solution = tweak(solution.copy(),mutation_prob)
        new_cost = tsp_cost(new_solution)
        current_cost = tsp_cost(solution)
        full_history.append(-new_cost)
        
        #if solution is better, accept it
        if new_cost < current_cost:
            solution = new_solution
            bettering_history.append(-new_cost)
        #else, accept it with a certain probability
        else:
            temperature = max(0.01, 1 - len(full_history) / NSTEPS)
            acceptance_probability = np.exp((current_cost - new_cost) / temperature)
            if acceptance_probability > np.random.rand():
                #Save last solution that was better than the current one
                top_cost = -tsp_cost(solution)
                top_solution = solution.copy()
                solution = new_solution
        
        bettering_history.append(-tsp_cost(solution))
        
    if top_cost < -tsp_cost(solution):
        top_solution = solution.copy()
        top_cost = -tsp_cost(solution)
    
    
    return full_history, bettering_history, top_cost, last_bettering, top_solution

Plotting Function

In [412]:
def plot_history(history, bettering_history, labels: list):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    ax.plot(history, label=labels[0])
    ax.plot(bettering_history, label=labels[1])
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Cost")
    ax.legend()
    plt.show()
    


## Genetic Algorithm


In [413]:
def inver_over_crossover(parent1: np.ndarray, parent2: np.ndarray):
    child = []
    n = len(parent1)
    visited = set()
    
    # Select a random starting index in parent1
    start = np.random.randint(0, n - 1)
    child.append(int(parent1[start]))
    visited.add(int(parent1[start]))
    # Find the corresponding city in parent2 and select the next one
    idx2 = np.where(parent2 == parent1[start])[0][0]
    idx2 = (idx2 + 1) % n
    child.append(int(parent2[idx2]))
    visited.add(int(parent2[idx2]))
    # Find index of parent2[idx2] in parent1
    idx1 = np.where(parent1 == parent2[idx2])[0][0]
    # Preserve edge of the first parent => meaning from parent1[idx1] to parent1[start]
    #  but inverse order of the cities
    index = ( idx1 + 1 ) % n
    idx1 = (idx1 - 1) % n
    while parent1[idx1] not in visited:
        child.append(int(parent1[idx1]))
        visited.add(int(parent1[idx1]))
        idx1 = (idx1 - 1) % n
    
    # Complete set of cities, preserving parent1
    while parent1[index] not in visited:
        child.append(int(parent1[index]))
        visited.add(int(parent1[index]))
        index = (index + 1) % n

    assert len(child) == n
    assert set(child) == set(range(n))

    #Return the child as a numpy array
    return np.array(child)

In [414]:
#try inver over crossover
"""
parent1 = random_genotype()
parent2 = random_genotype()
child = inver_over_crossover(parent1, parent2)
"""

'\nparent1 = random_genotype()\nparent2 = random_genotype()\nchild = inver_over_crossover(parent1, parent2)\n'

Generational (population_size, population_size+1or2, population_size)

In [415]:

def genetic_algorithm(NSTEPS, population_size, starting_solution):
    population = []
    if(starting_solution is not None):
        population.append(starting_solution)
        for _ in range(population_size):
            temp = starting_solution.copy()
            population.append(single_swap_mutation(temp))
    else:
        population = [random_genotype() for _ in range(population_size)]
    

    top_cost = tsp_cost(min(population, key=tsp_cost))
    last_bettering = 0
    print("Starting best cost: ", top_cost)
    last_bettering = 0
    genetic_operator = [multi_swap_mutation, inver_over_crossover]
    for step in tqdm(range(NSTEPS)):
        # Adjust mutation and crossover probabilities
        if step - last_bettering > NSTEPS / 10:
            mutation_prob = 0.9
            crossover_prob = 0.1
        else:
            mutation_prob = 0.1
            crossover_prob = 0.9

        # Choose the genetic operator
        operator = np.random.choice(genetic_operator, p=[mutation_prob, crossover_prob])

        # Select the correct number of parents (1 parent for mutation, 2 parents for crossover)
        if operator == multi_swap_mutation:
            nparents = 1
        else:
            nparents = 2

        # Parent selection (Tournament selection)
        parents = []
        for _ in range(nparents):
            tournament_indices = np.random.choice(len(population), 10, replace=False)
            tournament = [np.copy(population[i]) for i in tournament_indices]
            best = min(tournament, key=tsp_cost)
            parents.append(best)

        # Generate offspring
        if operator == multi_swap_mutation:
            # 10 mutated versions of the parent
            offspring = [operator(parents[0].copy()) for _ in range(10)]
            population.extend(offspring)
        else:
            offspring = operator(*parents)
            population.append(offspring)  # Add to population

        # Survivor selection (deterministic replacement)
        population = sorted(population, key=tsp_cost)[:population_size]

        # Update best cost
        top_cost = tsp_cost(population[0])

        # Update last bettering
        if top_cost < last_bettering:
            last_bettering = step
    top_cost = min(tsp_cost(ind) for ind in population)
    tsp = min(population, key=tsp_cost)    

    return tsp, top_cost, last_bettering

In [416]:
#Run the genetic algorithm
tsp, top_cost, last_bettering = genetic_algorithm(10000, 100, None)
print_tsp(tsp, top_cost)


Starting best cost:  15259.32585798163


100%|██████████| 10000/10000 [00:45<00:00, 219.50it/s]
INFO:root:result: Found a path of 46 steps, total length 4949.89km
DEBUG:root:step: Palermo -> Cagliari (390.28km)
DEBUG:root:step: Cagliari -> Sassari (173.89km)
DEBUG:root:step: Sassari -> Taranto (734.86km)
DEBUG:root:step: Taranto -> Bari (77.63km)
DEBUG:root:step: Bari -> Andria (50.18km)
DEBUG:root:step: Andria -> Foggia (67.42km)
DEBUG:root:step: Foggia -> Pescara (156.30km)
DEBUG:root:step: Pescara -> Ancona (139.24km)
DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Trieste (191.98km)
DEBUG:root:step: Trieste -> Venice (115.09km)
DEBUG:root:step: Venice -> Padua (36.07km)
DEBUG:root:step: Padua -> Vicenza (30.13km)
DEBUG:root:step: Vicenza -> Trento (67.35km)
DEBUG:root:step: Trento -> Bolzano (49.94km)
DEBUG:root:step: Bolzano -> Bergamo (157.42km)
DEBUG:root:step: Ber

In [ ]:
#Run the simulated annealing
print(" SIMULATED ANNEALING ")
full_history, bettering_history, top_cost, last_bettering, tsp = simulated_annealing(random_genotype(), [], [], 10000, 0.9)
plot_history(full_history, bettering_history, ["Full history", "Bettering history"])
print_tsp(tsp, -top_cost)

 SIMULATED ANNEALING 


AttributeError: 'numpy.ndarray' object has no attribute 'pop'

In [ ]:

#Run Greedy2
print_tsp(tsp_greedy2, tsp_cost(tsp_greedy2))